In [26]:
import os, sys
import numpy as np
import pandas as pd
import tensorflow as tf
import joblib

# notebook/에서 실행해도 루트(data, model이 있는 폴더) 기준으로 잡기
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

data_path = os.path.join(PROJECT_ROOT, "data")
model_path = os.path.join(PROJECT_ROOT, "model")
movielens_dir_nm = "ml-1m"

# 하이퍼파라미터(학습 때와 동일해야 함)
dropout = 0.4
embed_dim = 16

from autoint import AutoIntModel  # predict_model 안 쓰면 import 안 해도 됨

print("cwd:", os.getcwd())
print("PROJECT_ROOT:", PROJECT_ROOT)
print("data_path:", data_path)
print("model_path:", model_path)


cwd: c:\Users\소현진\projects\AutoInt_MLP\notebook
PROJECT_ROOT: c:\Users\소현진\projects\AutoInt_MLP
data_path: c:\Users\소현진\projects\AutoInt_MLP\data
model_path: c:\Users\소현진\projects\AutoInt_MLP\model


In [27]:
print("field_dims exists:", os.path.exists(os.path.join(data_path, "field_dims.npy")))
print("label_encoders exists:", os.path.exists(os.path.join(data_path, "label_encoders.pkl")))
print("ratings_prepro exists:", os.path.exists(os.path.join(data_path, movielens_dir_nm, "ratings_prepro.csv")))

field_dims = np.load(os.path.join(data_path, "field_dims.npy"))
label_encoders = joblib.load(os.path.join(data_path, "label_encoders.pkl"))

ratings_df = pd.read_csv(os.path.join(data_path, movielens_dir_nm, "ratings_prepro.csv"))
movies_df  = pd.read_csv(os.path.join(data_path, movielens_dir_nm, "movies_prepro.csv"))
user_df    = pd.read_csv(os.path.join(data_path, movielens_dir_nm, "users_prepro.csv"))

print("loaded shapes:", ratings_df.shape, movies_df.shape, user_df.shape)


field_dims exists: True
label_encoders exists: True
ratings_prepro exists: True


c:\Users\소현진\projects\AutoInt_MLP\.venv\Lib\site-packages\sklearn\base.py:463: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.6.1 when using version 1.8.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


loaded shapes: (1000209, 7) (3883, 10) (6040, 5)


In [31]:
weights_path = os.path.join(model_path, "autoint_model_weights.weights.h5")
print("weights exists:", os.path.exists(weights_path))
# 모델 생성
model = AutoIntModel(
    field_dims,
    embed_dim,
    att_layer_num=3,
    att_head_num=2,
    att_res=True,
    l2_reg_dnn=0,
    l2_reg_embedding=1e-5,
    dnn_use_bn=False,
    dnn_dropout=dropout,
    init_std=0.0001
)
# weights 로드 전에 모델을 한 번 호출해서 build
dummy_x = tf.constant([[0] * len(field_dims)], dtype=tf.int64)
_ = model(dummy_x)


# 가중치 로드
model.load_weights(weights_path)

pred = model.predict(dummy_x)
print("pred:", pred, "shape:", pred.shape)

assert pred.shape == (1, 1)
print("Model load & inference test passed.")



weights exists: True
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
pred: [[0.9005599]] shape: (1, 1)
Model load & inference test passed.
